In [3]:
from typing import List, Tuple, Optional, Union
from datetime import time
from pydantic import BaseModel, Field, validator, root_validator

# Input parameters
ROLE_NAME = {
    0: "Actor",
    1: "Shooting Crew",
    2: "Makeup Crew",
    3: "Lighting Crew",
    4: "Equipment"
}

ROLE_MEMBERS = {i:[] for i in ROLE_NAME.keys()}

class Task(BaseModel):
    id: int  # Add this line
    estimated_duration: int
    location: List[str] = []
    description: str = ""
    dependencies: List[Union[int, Tuple[int, int]]] = []
    time_of_day: List[Tuple[time, time]] = [(time(0, 0), time(23, 59))]
    members: List[int] = [],
    roles: List[Tuple[int, int]] = []
     
    @root_validator(pre=True)
    def check_members_or_roles(cls, values):
        if not values.get('members') and not values.get('roles'):
            raise ValueError("Task must have either members or roles specified.")
        return values

    def __str__(self):
        return f"Task({str(self.id)},{str(self.estimated_duration)},{str(self.location)},\
{str(self.description)},{str(self.dependencies)},{str(self.time_of_day)},{str(self.members)},{str(self.roles)})"

class Member(BaseModel):
    id: int
    name: str
    rate: int
    ot: int
    role: int  # Role ID
    blocked_timeslots: List[Tuple[time, time]] = Field(default_factory=list)
    transportation_speed: float = 0.5  # scale from 0 to 1, lower is faster
    working_hours: List[time] = Field(default_factory=list)
    assigned_tasks: List = Field(default_factory=list)
    schedule: List = Field(default_factory=list)
    
    def model_post_init(self, __context):
        """This runs after the model is initialized"""
        self._set_role()
    
    def __str__(self):
        return f"Member({str(self.id)},{str(self.name)},{str(self.rate)},{str(self.ot)},{str(self.role)},blocked_timeslots={str(self.blocked_timeslots)},transportation_speed={str(self.transportation_speed)})"

    def _set_role(self):
        global ROLE_MEMBERS
        if self.role not in ROLE_MEMBERS:
            ROLE_MEMBERS[self.role] = []
        ROLE_MEMBERS[self.role].append(self.id)

    def get_role(self) -> str:
        global ROLE_NAME
        return ROLE_NAME[self.role]
    
    def is_available_on(self, start_time: time, end_time: time) -> bool:
        """Check if the actor is available on the given date."""
        return not any((start <= start_time < end) or (start <= end_time < end) for start, end in self.blocked_timeslots)
    
    def block_time(self, start_time: time, end_time: time):
        """Block the time slot for the actor."""
        if self.working_hours: 
            self.working_hours = [min(self.working_hours[0], start_time), max(self.working_hours[1], end_time)]
        else: 
            self.working_hours = [start_time, end_time]
        self.blocked_timeslots.append((start_time, end_time))

In [4]:
from typing import List, Tuple, Union
from gurobipy import Model, GRB, quicksum
from collections import defaultdict
import pandas as pd
import numpy as np

In [5]:
class Optimizer:
    def __init__(self, tasks: List[Task], members: List[Member], ot_hours: int = 8):
        """
        Initializes the Optimizer class.

        Args:
            tasks (List[Task]): A list of tasks to be scheduled.
            members (List[Member]): A list of members available to work on the tasks.
            ot_hours (int, optional): The number of regular working hours per member before changing to a predefined OT rate. Defaults to 8.
        """
        self.tasks = tasks
        self.members = members
        self.ot_hours = ot_hours
        self.optimized_cost = np.inf
        self.best_solution = None
        self.time_unit = 15  # Minutes per time slot

    def get_by_id(self, type: Union[List[Task], List[Member]], id: int):
        """Get an object by its ID."""
        return next((obj for obj in type if obj.id == id), None)  
    
    def schedule_tasks(self, get_all_solutions=False, timeout=30, num_workers=8):
        """Schedule tasks using Gurobi solver."""
        # Initialize Gurobi model
        model = Model("TaskScheduler")
        model.setParam('TimeLimit', timeout)
        model.setParam('Threads', num_workers)
        
        # Calculate horizon (number of time slots in a day)
        horizon = int(1440 / self.time_unit)  # 24 hours = 1440 minutes
        
        # Create task variables
        task_vars = {}
        for task in self.tasks:
            # Start variables for each possible start time
            task_vars[task.id] = {
                'start': model.addVar(vtype=GRB.INTEGER, lb=0, ub=horizon-1, name=f'start_{task.id}'),
                'end': model.addVar(vtype=GRB.INTEGER, lb=0, ub=horizon, name=f'end_{task.id}'),
                'duration': int(task.estimated_duration / self.time_unit),
                'task': task
            }
            
            # Add constraint for duration
            model.addConstr(
                task_vars[task.id]['end'] == task_vars[task.id]['start'] + task_vars[task.id]['duration'],
                name=f'duration_{task.id}'
            )
            
            # Time window constraints
            if task.time_of_day:
                # Create time window constraints
                time_window_constraints = []
                for start_time, end_time in task.time_of_day:
                    start_slot = int(self._time_to_minutes(start_time) / self.time_unit)
                    end_slot = int(self._time_to_minutes(end_time) / self.time_unit)
                    
                    # Binary variable for if task is in this window
                    in_window = model.addVar(vtype=GRB.BINARY, name=f'task{task.id}_in_window_{start_slot}')
                    
                    # Add window constraints
                    model.addConstr(
                        (in_window == 0) | (task_vars[task.id]['start'] >= start_slot),
                        name=f'window_start_{task.id}_{start_slot}'
                    )
                    model.addConstr(
                        (in_window == 0) | (task_vars[task.id]['end'] <= end_slot),
                        name=f'window_end_{task.id}_{end_slot}'
                    )
                    
                    time_window_constraints.append(in_window)
                
                # Task must be in at least one time window
                model.addConstr(
                    quicksum(time_window_constraints) >= 1,
                    name=f'in_some_window_{task.id}'
                )

        # Handle dependencies
        for task in self.tasks:
            if task.dependencies:
                for dep in task.dependencies:
                    if isinstance(dep, tuple):
                        # OR dependency - at least one predecessor must be completed
                        or_constraints = []
                        for pred_id in dep:
                            # Binary variable for if this predecessor is satisfied
                            pred_satisfied = model.addVar(vtype=GRB.BINARY, name=f'pred_{pred_id}_satisfied_for_{task.id}')
                            
                            # Add constraint that if satisfied, predecessor end <= successor start
                            model.addConstr(
                                (pred_satisfied == 0) | (task_vars[pred_id]['end'] <= task_vars[task.id]['start']),
                                name=f'or_dep_{pred_id}_to_{task.id}'
                            )
                            
                            or_constraints.append(pred_satisfied)
                        
                        # At least one predecessor must be satisfied
                        model.addConstr(
                            quicksum(or_constraints) >= 1,
                            name=f'or_dep_atleast_one_{task.id}'
                        )
                    else:
                        # AND dependency - simple constraint (predecessor end <= successor start)
                        model.addConstr(
                            task_vars[dep]['end'] <= task_vars[task.id]['start'],
                            name=f'and_dep_{dep}_to_{task.id}'
                        )

        # Handle member assignments and constraints
        member_assignments = defaultdict(list)
        member_task_vars = {}
        
        for task in self.tasks:
            task_data = task_vars[task.id]
            
            # Handle role-based assignments
            for role_id, req_count in task.roles:
                role_assignments = []
                
                for member_id in ROLE_MEMBERS[role_id]:
                    # Binary variable for if member is assigned to this task
                    assigned = model.addVar(vtype=GRB.BINARY, name=f'role_{member_id}_assigned_to_{task.id}')
                    role_assignments.append(assigned)
                    
                    member_assignments[member_id].append((task.id, assigned))
                    
                    # Store this assignment variable for later use
                    if task.id not in member_task_vars:
                        member_task_vars[task.id] = {}
                    member_task_vars[task.id][member_id] = assigned
                    
                    # Handle blocked timeslots for this member
                    member = self.get_by_id(self.members, member_id)
                    self._add_blocked_time_constraints(model, member, task_data, assigned)
                
                # Enforce required number of members for this role
                model.addConstr(
                    quicksum(role_assignments) == req_count,
                    name=f'role_{role_id}_count_for_{task.id}'
                )
            
            # Handle direct member assignments
            for member_id in task.members:
                # These members must be assigned to the task
                if task.id not in member_task_vars:
                    member_task_vars[task.id] = {}
                
                assigned = model.addVar(vtype=GRB.BINARY, name=f'direct_{member_id}_assigned_to_{task.id}')
                model.addConstr(assigned == 1, name=f'must_assign_{member_id}_to_{task.id}')
                
                member_task_vars[task.id][member_id] = assigned
                member_assignments[member_id].append((task.id, assigned))
                
                # Handle blocked timeslots
                member = self.get_by_id(self.members, member_id)
                self._add_blocked_time_constraints(model, member, task_data, assigned)

        # No-overlap constraints for each member
        for member_id, assignments in member_assignments.items():
            # For each pair of tasks this member could be assigned to
            for i, (task1_id, assigned1) in enumerate(assignments):
                for j, (task2_id, assigned2) in enumerate(assignments[i+1:], i+1):
                    # If both are assigned, they can't overlap
                    task1 = task_vars[task1_id]
                    task2 = task_vars[task2_id]
                    
                    # Binary variable for if task1 comes before task2
                    task1_before_task2 = model.addVar(vtype=GRB.BINARY, name=f'task{task1_id}_before_task{task2_id}_for_{member_id}')
                    
                    # If both assigned and task1 before task2, then task1 end <= task2 start
                    model.addConstr(
                        (assigned1 == 0) | (assigned2 == 0) | (task1_before_task2 == 0) | (task1['end'] <= task2['start']),
                        name=f'no_overlap_{task1_id}_before_{task2_id}_for_{member_id}'
                    )
                    
                    # If both assigned and task2 before task1, then task2 end <= task1 start
                    model.addConstr(
                        (assigned1 == 0) | (assigned2 == 0) | (task1_before_task2 == 1) | (task2['end'] <= task1['start']),
                        name=f'no_overlap_{task2_id}_before_{task1_id}_for_{member_id}'
                    )

        # Calculate cost objective
        total_cost = self._calculate_costs(model, member_assignments, task_vars)
        model.setObjective(total_cost, GRB.MINIMIZE)
        
        # Solve the model
        model.optimize()
        
        # Process results
        if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.SOLUTION_LIMIT] and model.SolCount > 0:
            return self._process_results(model, task_vars, member_task_vars, get_all_solutions)
        else:
            print(f"No solution found. Status: {model.status}")
            return None

    def _add_blocked_time_constraints(self, model, member, task_data, assigned_var):
        """Helper method to add blocked time constraints for a member"""
        if not member.blocked_timeslots:
            return
            
        for start_time, end_time in member.blocked_timeslots:
            block_start = int(self._time_to_minutes(start_time) / self.time_unit)
            block_end = int(self._time_to_minutes(end_time) / self.time_unit)
            
            # Binary variables for if task is before or after blocked period
            before_blocked = model.addVar(vtype=GRB.BINARY, name=f'mem{member.id}_task{task_data["task"].id}_before_{block_start}')
            after_blocked = model.addVar(vtype=GRB.BINARY, name=f'mem{member.id}_task{task_data["task"].id}_after_{block_end}')
            
            # If task is before blocked period, then task end <= block start
            model.addConstr(
                (before_blocked == 0) | (task_data['end'] <= block_start),
                name=f'mem{member.id}_task{task_data["task"].id}_before_block_{block_start}'
            )
            
            # If task is after blocked period, then task start >= block end
            model.addConstr(
                (after_blocked == 0) | (task_data['start'] >= block_end),
                name=f'mem{member.id}_task{task_data["task"].id}_after_block_{block_end}'
            )
            
            # If member is assigned to task, task must be either before or after blocked period
            model.addConstr(
                (assigned_var == 0) | (before_blocked + after_blocked >= 1),
                name=f'mem{member.id}_task{task_data["task"].id}_avoid_block_{block_start}_{block_end}'
            )

    def _calculate_costs(self, model, member_assignments, task_vars):
        """Calculate total cost based on member assignments and schedule span"""
        total_cost = 0
        
        for member_id, assignments in member_assignments.items():
            if not assignments:
                continue
                
            member = self.get_by_id(self.members, member_id)
            
            # Calculate earliest start and latest end for this member's assigned tasks
            task_start_times = []
            task_end_times = []
            
            for task_id, assigned in assignments:
                task_data = task_vars[task_id]
                
                # Create conditional start/end variables
                # If assigned, use task start/end; otherwise use dummy values
                conditional_start = model.addVar(lb=0, ub=1440, name=f'cond_start_{member_id}_{task_id}')
                conditional_end = model.addVar(lb=0, ub=1440, name=f'cond_end_{member_id}_{task_id}')
                
                # Big-M constraints for conditional values
                big_M = 1440  # Maximum minutes in a day
                
                # If assigned, conditional_start = task_start, else = big_M
                model.addConstr(
                    conditional_start >= task_data['start'],
                    name=f'cond_start_lb_{member_id}_{task_id}'
                )
                model.addConstr(
                    conditional_start <= task_data['start'] + big_M * (1 - assigned),
                    name=f'cond_start_ub_{member_id}_{task_id}'
                )
                model.addConstr(
                    conditional_start >= big_M * (1 - assigned),
                    name=f'cond_start_dummy_{member_id}_{task_id}'
                )
                
                # If assigned, conditional_end = task_end, else = 0
                model.addConstr(
                    conditional_end <= task_data['end'],
                    name=f'cond_end_ub_{member_id}_{task_id}'
                )
                model.addConstr(
                    conditional_end >= task_data['end'] - big_M * (1 - assigned),
                    name=f'cond_end_lb_{member_id}_{task_id}'
                )
                model.addConstr(
                    conditional_end <= big_M * assigned,
                    name=f'cond_end_dummy_{member_id}_{task_id}'
                )
                
                task_start_times.append(conditional_start)
                task_end_times.append(conditional_end)
            
            # Find earliest start and latest end
            earliest_start = model.addVar(lb=0, ub=1440, name=f'earliest_start_{member_id}')
            latest_end = model.addVar(lb=0, ub=1440, name=f'latest_end_{member_id}')
            
            # Earliest start is minimum of all start times
            for start_var in task_start_times:
                model.addConstr(
                    earliest_start <= start_var,
                    name=f'min_start_{member_id}_{start_var.VarName}'
                )
            
            # Latest end is maximum of all end times
            for end_var in task_end_times:
                model.addConstr(
                    latest_end >= end_var,
                    name=f'max_end_{member_id}_{end_var.VarName}'
                )
            
            # Calculate time span
            time_span = model.addVar(lb=0, ub=1440, name=f'time_span_{member_id}')
            model.addConstr(
                time_span == latest_end - earliest_start,
                name=f'time_span_calc_{member_id}'
            )
            
            # Calculate regular hours and overtime
            regular_hours_limit = int(self.ot_hours * 60 / self.time_unit)
            regular_hours = model.addVar(lb=0, ub=regular_hours_limit, name=f'reg_hours_{member_id}')
            overtime = model.addVar(lb=0, ub=1440, name=f'overtime_{member_id}')
            
            # Regular hours is minimum of time span and regular hours limit
            model.addConstr(
                regular_hours <= time_span,
                name=f'reg_hours_limit1_{member_id}'
            )
            model.addConstr(
                regular_hours <= regular_hours_limit,
                name=f'reg_hours_limit2_{member_id}'
            )
            
            # Overtime is time span minus regular hours
            model.addConstr(
                overtime == time_span - regular_hours,
                name=f'overtime_calc_{member_id}'
            )
            
            # Calculate cost
            rate = int(member.rate * self.time_unit / 60)  # Convert hourly rate to rate per time unit
            ot_rate = int(member.ot * self.time_unit / 60)  # Convert hourly OT rate to rate per time unit
            
            member_cost = model.addVar(lb=0, ub=GRB.INFINITY, name=f'cost_{member_id}')
            model.addConstr(
                member_cost == regular_hours * rate + overtime * ot_rate,
                name=f'cost_calc_{member_id}'
            )
            
            total_cost += member_cost
            
        return total_cost

    def _process_results(self, model, task_vars, member_task_vars, get_all_solutions=False):
        """Process the results from the solved model"""
        if get_all_solutions:
            # Gurobi doesn't easily support getting all solutions
            # We'll just return the best solution found
            print("Warning: Returning only the best solution. Gurobi doesn't easily support retrieving all solutions.")
        
        solution = {}
        for task_id, task_data in task_vars.items():
            start_minutes = int(task_data['start'].X * self.time_unit)
            solution[task_id] = start_minutes
        
        # Set member schedules
        sol_member_schedule = defaultdict(list)
        for task_id, member_dict in member_task_vars.items():
            task_data = task_vars[task_id]
            for member_id, assigned_var in member_dict.items():
                if assigned_var.X > 0.5:  # If member is assigned to task
                    start_minutes = int(task_data['start'].X * self.time_unit)
                    end_minutes = int(task_data['end'].X * self.time_unit)
                    sol_member_schedule[member_id].append({
                        'task': task_id,
                        'start': self._minutes_to_time(start_minutes),
                        'end': self._minutes_to_time(end_minutes)
                    })
        
        self.optimized_cost = model.objVal
        self.best_solution = [(solution, sol_member_schedule)]
        return self.best_solution
    
    def _minutes_to_time(self, minutes):
        """Convert minutes since midnight to HH:MM format"""
        return f"{minutes//60:02d}:{minutes%60:02d}"
    
    def _time_to_minutes(self, t: time) -> int:
        """Convert time object to minutes since midnight."""
        return t.hour * 60 + t.minute

In [6]:
import matplotlib.pyplot as plt

def export_schedules(res, model, export_location=''):
    for i, sol in enumerate(res):
        def visualize_schedule(solution_index, solution):
            # Prepare data for plotting
            tasks = []
            for member_id, member_tasks in solution[1].items():
                member_name = model.get_by_id(model.members, member_id).name
                for task in member_tasks:
                    task_obj = model.get_by_id(model.tasks, task['task'])
                    
                    # Convert start and end time to minutes
                    start_mins = sum(int(x) * y for x, y in zip(task['start'].split(':'), [60, 1]))
                    end_mins = sum(int(x) * y for x, y in zip(task['end'].split(':'), [60, 1]))
                    
                    tasks.append({
                        'Member': member_name,
                        'MemberID': member_id,
                        'Task': f"Task {task['task']}",
                        'Task Description': task_obj.description,
                        'Start': start_mins,
                        'Duration': end_mins - start_mins,
                        'End': end_mins,
                    })
            
            # Convert to DataFrame for easier plotting
            df = pd.DataFrame(tasks)
            
            # Sort members by name
            members = sorted(df['Member'].unique())
            
            # Create figure and axes
            fig, ax = plt.subplots(figsize=(15, 12))
            
            # Define color palette
            task_colors = plt.cm.Set3(np.linspace(0, 1, len(df['Task'].unique())))
            
            # Plot tasks as rectangles
            for idx, task in df.iterrows():
                x_pos = members.index(task['Member'])
                task_idx = task['Task'].split()[1]  # Get the task number
                rect = Rectangle((x_pos - 0.25, task['Start']), 0.5, task['Duration'],
                                 facecolor=task_colors[int(task_idx) % len(df['Task'].unique())],
                                 label=f"Task {task_idx} - {task['Task Description']}")
                ax.add_patch(rect)
                ax.text(x_pos, task['Start'] + task['Duration']/2,
                        f'Task\n{task_idx}', ha='center', va='center', rotation=0)
            
            # Customize the plot
            ax.set_xlim(-0.5, len(members) - 0.5)
            ax.set_ylim(top=df['Start'].min(), bottom=(df['Start'] + df['Duration']).max())
            ax.set_xticks(range(len(members)))
            ax.set_xticklabels(members)
            
            # Add gridlines
            ax.grid(True, axis='y', alpha=0.3)
            
            # Format y-axis as time
            yticks = np.arange(df['Start'].min()-15,(df['Start'] + df['Duration']).max()+30 , 15)
            ax.set_yticks(yticks)
            ax.set_yticklabels([f'{y//60:02d}:{y%60:02d}' for y in yticks])

            # Move y-axis ticks to the right side and adjust appearance
            ax.yaxis.set_label_position('right')
            ax.yaxis.tick_right()

            # Add legend
            handles, labels = [], []
            for patch in ax.patches:
                handles.append(patch)
                labels.append(patch.get_label())

            # Remove duplicates while preserving order
            unique_labels = list(dict.fromkeys(labels))
            unique_handles = [handles[labels.index(label)] for label in unique_labels]  # Get unique handles
            unique_pairs = list(zip(unique_labels, unique_handles))  # Create pairs of labels and handles
            
            # Sort by task number
            sorted_pairs = sorted(unique_pairs, 
                                key=lambda x: int(x[0].split()[1]))  # Sort by task number
            
            # Unzip the sorted pairs
            labels, handles = zip(*sorted_pairs)
            ax.legend(handles, labels, 
                     loc='upper center', 
                     bbox_to_anchor=(0.5, -0.05),
                     fancybox=True, 
                     shadow=True, 
                     ncol=4,
                     title='Tasks')
          
            plt.title(f'Schedule Solution {solution_index + 1}')
            plt.tight_layout()
            
            # Save the timetable plot
            plt.savefig(export_location+f'/schedule_solution_{solution_index + 1}.png')
            plt.close()
            
            # Export to csv
            df_export = df.copy()
            df_export = df_export.sort_values(['Member', 'Start'])

            # Convert minutes back to time
            df_export['Start'] = df_export['Start'].apply(lambda x: f'{x//60:02d}:{x%60:02d}')
            df_export['End'] = df_export['End'].apply(lambda x: f'{x//60:02d}:{x%60:02d}')
            
            csv_filename = f'{export_location}/schedule_solution_{solution_index + 1}.csv'
            df_export.to_csv(csv_filename, index=False)
            print(f'Solution {solution_index + 1} saved to {export_location}')

        visualize_schedule(i, sol)

In [7]:
class Optimizer(Optimizer):
    def __init__(self, tasks: List[Task], members: List[Member], ot_hours: int = 8):
        super().__init__(tasks, members, ot_hours)

    def export_schedule(self, export_location=''):
        export_schedules(self.best_solution, self, export_location=export_location)

In [8]:
rs = []
Jacob = Member(0, "Jacob", [(time(0, 0), time(8, 0)), (time(18, 0), time(23, 59))], 100, 150, 0, 0.5)
Vicky = Member(1, "Vicky", [(time(0, 0), time(8, 0)), (time(18, 0), time(23, 59))], 300, 350, 0, 0.5)
Monita = Member(2, "Monita", [(time(0, 0), time(8, 0)), (time(18, 0), time(23, 59))], 200, 250, 0, 0.5)
Ray = Member(3, "Ray", [(time(0, 0), time(8, 0)), (time(18, 0), time(23, 59))], 600, 750, 0, 0.5)
John = Member(4, "John", [(time(0, 0), time(12, 0)), (time(21, 0), time(23, 59))], 200, 250, 1, 0.5)
Sara = Member(5, "Sara", [(time(0, 0), time(11, 0)), (time(21, 0), time(23, 59))], 200, 250, 1, 0.5)
Raj = Member(6, "Raj", [(time(0, 0), time(8, 0)), (time(21, 0), time(23, 59))], 200, 250, 1, 0.5)
Kumar = Member(7, "Kumar", [(time(0, 0), time(8, 0)), (time(17, 0), time(23, 59))], 200, 250, 1, 0.5)
Rahul = Member(8, "Rahul", [(time(0, 0), time(8, 0)), (time(18, 0), time(23, 59))], 300, 350, 1, 0.5)
Rajesh = Member(9, "Rajesh", [(time(0, 0), time(23, 59))], 300, 350, 1, 0.5)
Ramesh = Member(10, "Ramesh", [(time(0, 0), time(8, 0)), (time(21, 0), time(23, 59))], 450, 450, 1, 0.5)
Rajat = Member(11, "Rajat", [(time(0, 0), time(8, 0)), (time(21, 0), time(23, 59))], 450, 450, 1, 0.5)
Bonnie = Member(12, "Bonnie", [(time(0, 0), time(8, 0)), (time(21, 0), time(23, 59))], 450, 550, 2, 0.5)
Sheila = Member(13, "Sheila", [(time(0, 0), time(8, 0)), (time(21, 0), time(23, 59))], 450, 550, 2, 0.5)
Sandy = Member(14, "Sandy", [(time(0, 0), time(14, 0)), (time(21, 0), time(23, 59))], 250, 300, 2, 0.5)
Camera1 = Member(15, "Camera1", [(time(0, 0), time(9, 0)), (time(20, 0), time(23, 59))], 50, 50, 4, 0.5)
Camera2 = Member(16, "Camera2", [(time(8, 0), time(14, 0)), (time(23, 0), time(23, 59))], 50, 50, 4, 0.5)
Camera3 = Member(17, "Camera3", [(time(0, 0), time(8, 0)), (time(21, 0), time(23, 59))], 50, 50, 4, 0.5)
Camera4 = Member(18, "Camera4", [(time(0, 0), time(8, 0)), (time(21, 0), time(23, 59))], 50, 50, 4, 0.5)
Camera5 = Member(19, "Camera5", [(time(0, 0), time(8, 0)), (time(21, 0), time(23, 59))], 50, 50, 4, 0.5)

members = [Jacob, Vicky, Monita, Ray, John, Sara, Raj, Kumar, Rahul, Rajesh, Ramesh, Rajat, Bonnie, Sheila, 
           Sandy, Camera1, Camera2, Camera3, Camera4, Camera5]
JacobMakeup = Task(0, ["Parking Lot Near Loading Bay"], 30, members=[0], roles=[(2, 1)])
SetupLightingParkingLot = Task(1, ["Parking Lot Near Loading Bay"], 75, roles=[(1, 5), (4, 2)], members=[15])
VickyMakeup = Task(2, ["Parking Lot Near Loading Bay"], 75, members=[1, 13])
JacobShot = Task(3, ["Parking Lot Near Loading Bay"], 60, members=[0, 15], roles=[(1, 5),(4, 2)], dependencies=[(0, 5), 1])
VickyPhotoshoot = Task(4, ["The Galleria", "Studio"], 60, members=[1], roles=[(1, 2),(4, 2)], dependencies=[20, (2, 8)])
JacobMakeupTouchUp = Task(5, ["Parking Lot Near Loading Bay"], 15, members=[0], roles=[(2, 1)], dependencies=[0, (3, 7)])
SetupLightingExt = Task(6, ["Parking Lot Near Loading Bay"], 60, roles=[(1, 5), (4, 2)], members=[15])
JacobPhotoshoot = Task(7, ["The Galleria", "Studio"], 60, members=[0], roles=[(1, 2),(4, 2)], dependencies=[20, (0, 5)])
VickyMakeupTouchUp = Task(8, ["Parking Lot Near Loading Bay", "The Galleria", "Studio"], 15, members=[1,13], roles=[(2, 1)], dependencies=[2, (4, 9)])
VickyShot = Task(9, ["Parking Lot Near Loading Bay"], 30, members=[1, 15], roles=[(1, 5),(4, 2)], dependencies=[(2, 8), 6])
# Lunch = Task(10, ["Cathay City Loading Bay"], 60, roles=[(1, 5), (4, 2)], members=[15])
RayMakeup = Task(10, ["Entrance"], 45, members=[3], roles=[(2, 1)])
SetupLightingEntrance = Task(11, ["Entrance"], 105, roles=[(1, 5), (4, 2)], members=[15])
MonitaMakeup = Task(12, ["The Galleria", "Studio"], 75, members=[2, 13])
RayShot = Task(13, ["Entrance"], 60, members=[3, 15], roles=[(1, 5), (4, 2)], dependencies=[11, (10, 15)])
MonitaPhotoshoot = Task(14, ["The Galleria", "Studio"], 60, members=[2], roles=[(1, 2),(4, 2)], dependencies=[20, (12, 18)])
RayMakeupTouchUp = Task(15, ["Entrance"], 15, members=[3], roles=[(2, 1)], dependencies=[10, (13, 17)])
SetupLightingStreet = Task(16, ["The Street"], 45, roles=[(1, 5), (4, 2)], members=[15])
RayPhotoshoot = Task(17, ["The Galleria", "Studio"], 60, members=[3], roles=[(1, 2),(4, 2)], dependencies=[20])
MonitaMakeupTouchUp = Task(18, ["Parking Lot Near Loading Bay", "The Galleria", "Studio"], 30, members=[1,13], roles=[(2, 1)], dependencies=[2, (4, 9)])
MonitaShot = Task(19, ["Parking Lot Near Loading Bay"], 60, members=[2, 15], roles=[(1, 5),(4, 2)], dependencies=[8, (4, 9)])
SetupPhotoArea = Task(20, ["The Galleria", "Studio"], 30, roles=[(1, 2), (4, 2)])

tasks = [JacobMakeup, SetupLightingParkingLot, VickyMakeup, JacobShot, VickyPhotoshoot, JacobMakeupTouchUp, SetupLightingExt, JacobPhotoshoot, VickyMakeupTouchUp, VickyShot, 
         RayMakeup, SetupLightingEntrance, MonitaMakeup, RayShot, MonitaPhotoshoot, RayMakeupTouchUp, SetupLightingStreet, RayPhotoshoot, MonitaMakeupTouchUp, MonitaShot, SetupPhotoArea]

model = Optimizer(tasks, members)
res = model.schedule_tasks(timeout=60)
print('The optimized cost is: ', model.optimized_cost)
export_schedules(res, model, location=r'/Users/newtonlouey/Documents/Newton/Study U/FYP/GITHUB_CLONE/production_scheduler/newton')

TypeError: __init__() takes exactly 1 positional argument (8 given)